# Importing Data

This part of the project uses very helpful data and functions from the following sources:

- https://www.kaggle.com/code/gavinjpng/fpl-prediction-and-selection/notebook (cleaning and helper functions, see utils)
- https://github.com/vaastav/Fantasy-Premier-League (player and gw data)
- https://github.com/solpaul/fpl-prediction/ (team id cross referencing)

Vaastav has created one of the most comprehensive historical datasets for FPL purposes, many researchers and hobbyists use this source as historical data is hard to find, unless you archived it at the time of release, which I had unfortunately not.

In [5]:
import pandas as pd
import Utils

Wrt the cell below, the player dataframes contain summary statistics for each player at the end of the season, while the gameweek dataframes break down each player performance by gameweek. The formatting of the data changes from one season to the next, which is why encoding methods were specified/varied when importing some of the data.

In [6]:
#summary stats for each season
players_1617 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/players_raw.csv')
players_1718 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/players_raw.csv')
players_1819 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/players_raw.csv')
players_1920 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/players_raw.csv')
players_2021 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2020-21/players_raw.csv')
players_2122 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2021-22/players_raw.csv')

#gameweek breakdown for each player
gws_1617 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/merged_gw.csv',encoding="latin")
gws_1718 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/merged_gw.csv',encoding="latin")
gws_1819 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/merged_gw.csv',encoding="latin")
gws_1920 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/merged_gw.csv',encoding="utf-8")
gws_2021 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2020-21/gws/merged_gw.csv',encoding="utf-8")
gws_2122 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2021-22/gws/merged_gw.csv',encoding="utf-8")

In [7]:
team_ids = pd.read_csv('https://raw.githubusercontent.com/solpaul/fpl-prediction/master/fpl_predictor/data/teams.csv')

Here you can see the team_code is quite arbitrary, and the team rank changes from year to year.

In [8]:
team_ids.head()

,team,team_code,team_1617,team_1718,team_1819,team_1920,team_2021,team_2122,team_2223
0,Arsenal,3,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,Bournemouth,91,2.0,2.0,2.0,3.0,NaN,NaN,3.0
2,Burnley,90,3.0,4.0,4.0,5.0,4.0,5.0,NaN
3,Chelsea,8,4.0,5.0,6.0,6.0,5.0,6.0,6.0
4,Crystal Palace,31,5.0,6.0,7.0,7.0,6.0,7.0,7.0


# Data Formatting and Cleaning

From the column names alone, we can already see some likely issues. 

- First we should check that all player codes are unique, while the player code doesn't matter in the grand scheme of things, it will help to check for duplicate entries.
- The gw dfs only have 'full name' rather than 'first' and 'second' name as found in the player dfs, we should change the player df
- Some categorical variables e.g. photo, special don't really have any meaning for our analysis. We should investigate which features have meaning (See eda notebook)
- In order to merge the dataframes, we need to denote which season each column is from.

## Adding the season

In [9]:
#to save time, we should concatenate each season df into one, same with gws
#but we should make note of which season the data came from first
years = ['1617','1718','1819','1920','2021','2122']
years_by_num = list(range(len(years)))

player_season_list = [players_1617,players_1718,players_1819,players_1920,players_2021,players_2122]
gw_season_list = [gws_1617,gws_1718,gws_1819,gws_1920,gws_2021,gws_2122]

for i in range(len(years)):
    player_season_list[i]['season'] = years[i] #adds season e.g. 1617...2122
    gw_season_list[i]['season'] = years[i]
    
    player_season_list[i]['season_num'] = years_by_num[i] #adds number e.g. 1,2,3...
    gw_season_list[i]['season_num'] = years_by_num[i]

In [10]:
#merging dfs
players = pd.concat(player_season_list)
all_gws = pd.concat(gw_season_list)

In [11]:
players.reset_index(inplace = True)
all_gws.reset_index(inplace = True)

## Adding position

In [12]:
#get this just from inspecting the data and from general knowledge, i.e we know Ospina is a GK, so 1 = GK
#players[['element_type','web_name']]
positions = {1: 'GK', 2: 'DEF', 3: 'MID', 4: 'FWD'}

#add in positional data
players['position'] = players.element_type.map(positions)

## Cleaning the names

In [13]:
players['full_name'] = players.apply(lambda x: Utils.combine_names(x.first_name, x.second_name), axis=1).str.lower()

In [14]:
#just checking if there are numbers in any names
for i in players.full_name:
    if i.isalnum() == True:
        print(i)

In order to merge any part of the player df to the gw df, we need to ensure the names match

In [15]:
all_gws.name.head()

0       Aaron_Cresswell
1          Aaron_Lennon
2          Aaron_Ramsey
3    Abdoulaye_Doucouré
4     Abdul Rahman_Baba
Name: name, dtype: object

In [16]:
players.sort_values('full_name').full_name

3474     aaron_connolly
2082     aaron_connolly
2711     aaron_connolly
2550    aaron_cresswell
4005    aaron_cresswell
             ...       
2222     çaglar_söyüncü
1672     çaglar_söyüncü
2919     çaglar_söyüncü
2662       ørjan_nyland
1996       ørjan_nyland
Name: full_name, Length: 4070, dtype: object

In [17]:
#make gw df the same format
all_gws['full_name'] = all_gws.name.str.replace(' ', '_')
all_gws['full_name'] = all_gws['full_name'].str.replace('-', '_')
all_gws['full_name'] = all_gws['full_name'].str.lower()
all_gws['full_name'] = all_gws['full_name'].str.replace('_\d+','', regex = True) #sometimes there is an underscore and a number at the end

In [18]:
for i in all_gws.full_name:
    if i.isalnum() == True:
        print(i)

In [19]:
count_names = all_gws.groupby(['full_name','season']).count()

The following cell tells us that there were probably three duplicates in the data who are now being counted as the same player.

In [20]:
print('The number of players in the players dataset is', len(players), 'and the number in the gw dataset is',len(count_names))

The number of players in the players dataset is 4070 and the number in the gw dataset is 4067


In [21]:
test_dups = players[players.duplicated(['full_name','season'], keep = False)]

The cell below tells us that we should just delete the duplicate names with 0 points.

In [22]:
test_dups[['full_name', 'team_code','position','total_points','minutes']]

,full_name,team_code,position,total_points,minutes
1473,danny_ward,97,MID,22,435
1669,danny_ward,13,GK,0,0
3001,ben_davies,14,DEF,0,0
3199,ben_davies,6,DEF,52,1340
3446,álvaro_fernández,94,GK,31,1080
3730,ben_davies,14,DEF,0,0
3805,álvaro_fernández,1,DEF,0,0
3921,ben_davies,6,DEF,105,2537


In [23]:
players = players.drop([1669,3001, 3805, 3730])

## Adding Team Name

In [24]:
players['player_team_name'] = players.team_code.map(team_ids.set_index('team_code').team)
all_gws_df = Utils.cleaned(players, all_gws, team_ids)

C:\Users\Tasha\Documents\Data Science\Final Dissertation Submission\Utils.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seasons['opponent_team_name'] = seasons.opponent_team.map(team_id.set_index(str('team_'+i)).team)


In [25]:
all_gws_df

,index,name,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,...,GW,season,season_num,position_x,team,xP,full_name,position_y,player_team_name,opponent_team_name
0,0,Aaron_Cresswell,0,0.0,0.0,0.0,0,0,0,0.0,...,1,1617,0,NaN,NaN,NaN,aaron_cresswell,DEF,West Ham United,Chelsea
1,524,Aaron_Cresswell,0,41.0,0.0,0.0,0,12,0,1.0,...,10,1617,0,NaN,NaN,NaN,aaron_cresswell,DEF,West Ham United,Everton
2,1122,Aaron_Cresswell,0,30.0,0.0,0.0,0,13,0,4.0,...,11,1617,0,NaN,NaN,NaN,aaron_cresswell,DEF,West Ham United,Stoke City
3,1721,Aaron_Cresswell,0,37.0,0.0,0.0,0,13,0,2.0,...,12,1617,0,NaN,NaN,NaN,aaron_cresswell,DEF,West Ham United,Tottenham Hotspur
4,2322,Aaron_Cresswell,0,24.0,0.0,0.0,0,7,0,4.0,...,13,1617,0,NaN,NaN,NaN,aaron_cresswell,DEF,West Ham United,Manchester United
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140277,25240,Joseph McGlynn,0,NaN,NaN,NaN,0,0,0,NaN,...,38,2122,5,FWD,Burnley,-0.5,joseph_mcglynn,FWD,Burnley,Newcastle United
140278,24678,Anthony Mancini,0,NaN,NaN,NaN,0,0,0,NaN,...,37,2122,5,MID,Burnley,1.1,anthony_mancini,MID,Burnley,Tottenham Hotspur
140279,24679,Anthony Mancini,0,NaN,NaN,NaN,0,0,0,NaN,...,37,2122,5,MID,Burnley,1.1,anthony_mancini,MID,Burnley,Aston Villa
140280,25018,Anthony Mancini,0,NaN,NaN,NaN,0,0,0,NaN,...,38,2122,5,MID,Burnley,-0.5,anthony_mancini,MID,Burnley,Newcastle United


In [26]:
#removing those same players from earlier

all_gws_df = all_gws_df.drop(all_gws_df[(all_gws_df.full_name == 'danny_ward') & (all_gws_df.player_team_name == 'Leicester City')].index)
all_gws_df = all_gws_df.drop(all_gws_df[(all_gws_df.full_name == 'ben_davies') & (all_gws_df.player_team_name == 'Liverpool')].index)
all_gws_df = all_gws_df.drop(all_gws_df[(all_gws_df.full_name == 'álvaro_fernández') & (all_gws_df.player_team_name == 'Manchester United')].index)

## Deleting unneccessary features

See EDA script, or alternatively the report for justification

In [27]:
#determined as not relavant to project
del all_gws_df['attempted_passes']
del all_gws_df['xP']
del all_gws_df['transfers_balance']

#unsure why this happened, probably have clashing position cols somewhere, but pos_y was correct, pos_x had NaNs
del all_gws_df['position_x']
all_gws_df.rename(columns = {'position_y': 'position'}, inplace = True)

#merged with player df, so can delete 
del all_gws_df['team']
del all_gws_df['opponent_team']
del all_gws_df['name']

## Adding opponent features

In [28]:
#add team points, useful when considering difficulty
all_gws_df['team_points']= all_gws_df.apply(lambda x: Utils.get_team_points(x.was_home, x.team_h_score, x.team_a_score), axis=1)
all_gws_df['opponent_points'] = all_gws_df.team_points.apply(lambda x: Utils.get_opponent_points(x))

# Make Historic Features

## All positions

In [29]:
features_to_use_last_yr = ['assists', 'bonus', 'bps', 'clean_sheets','creativity','dreamteam_count','goals_conceded',
                          'goals_scored','ict_index','influence','minutes','own_goals','penalties_missed',
                           'penalties_saved', 'points_per_game','red_cards','saves', 'transfers_in','transfers_out',
                          'yellow_cards']
team_feats_to_use = ['goals_conceded', 'goals_scored','team_points','opponent_points']

In [30]:
all_gws_df.columns

Index(['index', 'assists', 'big_chances_created', 'big_chances_missed',
       'bonus', 'bps', 'clean_sheets', 'clearances_blocks_interceptions',
       'completed_passes', 'creativity', 'dribbles', 'ea_index', 'element',
       'errors_leading_to_goal', 'errors_leading_to_goal_attempt', 'fixture',
       'fouls', 'goals_conceded', 'goals_scored', 'ict_index', 'id',
       'influence', 'key_passes', 'kickoff_time', 'kickoff_time_formatted',
       'loaned_in', 'loaned_out', 'minutes', 'offside', 'open_play_crosses',
       'own_goals', 'penalties_conceded', 'penalties_missed',
       'penalties_saved', 'recoveries', 'red_cards', 'round', 'saves',
       'selected', 'tackled', 'tackles', 'target_missed', 'team_a_score',
       'team_h_score', 'threat', 'total_points', 'transfers_in',
       'transfers_out', 'value', 'was_home', 'winning_goals', 'yellow_cards',
       'GW', 'season', 'season_num', 'full_name', 'position',
       'player_team_name', 'opponent_team_name', 'team_points',
  

In [31]:
hist_player_features = ['assists','big_chances_created','big_chances_missed','bonus', 'bps','clearances_blocks_interceptions',
                        'creativity', 'clean_sheets','goals_conceded','goals_scored', 'completed_passes', 'dribbles',
                        'errors_leading_to_goal', 'fouls','key_passes','open_play_crosses','own_goals','penalties_conceded',
                        'penalties_missed','penalties_saved','recoveries','red_cards','saves','yellow_cards', 'ict_index',
                        'influence','minutes','threat']

In [32]:
historical_players_df = Utils.historical_data(all_gws_df, hist_player_features, ['all',1,3,5]) 
#when doing transfers, may want to rerun this to get 7,9,10 etc average
historical_team_df = Utils.historical_team(historical_players_df, team_feats_to_use, ['all',1,3,5])

C:\Users\Tasha\Documents\Data Science\Final Dissertation Submission\Utils.py:100: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[hist_feature] = df.sort_values('round').groupby(['full_name',
C:\Users\Tasha\Documents\Data Science\Final Dissertation Submission\Utils.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[hist_feature] = df.sort_values('round').groupby(['full_name',


In [33]:
historical_team_df

,index,assists,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,creativity,...,opponent_team_name_opponent,opponent_points_team_opponent,last_all_opponent_points_team,opponent_last_all_opponent_points,last_1_opponent_points_team,opponent_last_1_opponent_points,last_3_opponent_points_team,opponent_last_3_opponent_points,last_5_opponent_points_team,opponent_last_5_opponent_points
0,0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,...,West Ham United,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,524,0,0.0,0.0,0,12,0,1.0,35.0,7.9,...,West Ham United,0.0,126.0,10.0,3.0,0.0,4.0,3.0,5.0,3.0
2,1122,0,0.0,0.0,0,13,0,4.0,23.0,1.9,...,West Ham United,1.0,134.0,10.0,0.0,0.0,1.0,0.0,2.0,3.0
3,1721,0,0.0,0.0,0,13,0,2.0,29.0,2.8,...,West Ham United,0.0,153.0,11.0,3.0,0.0,9.0,0.0,12.0,1.0
4,2322,0,0.0,0.0,0,7,0,4.0,16.0,1.7,...,West Ham United,1.0,177.0,12.0,1.0,1.0,2.0,1.0,6.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140163,25240,0,NaN,NaN,0,0,0,NaN,NaN,0.0,...,Burnley,0.0,413.0,266.0,3.0,1.0,9.0,1.0,15.0,7.0
140164,24678,0,NaN,NaN,0,0,0,NaN,NaN,0.0,...,Burnley,0.0,391.0,265.0,3.0,0.0,7.0,6.0,13.0,10.0
140165,24679,0,NaN,NaN,0,0,0,NaN,NaN,0.0,...,Burnley,1.0,394.0,265.0,3.0,0.0,9.0,3.0,13.0,9.0
140166,25018,0,NaN,NaN,0,0,0,NaN,NaN,0.0,...,Burnley,0.0,413.0,266.0,3.0,1.0,9.0,1.0,15.0,7.0


In [34]:
def get_last_season_overall(gw, overall, feats_to_add):
    for feat in feats_to_add:
        print(feat) #this will take a long time so progress check
        feats = []
        feature_name = 'last_yr_' + feat 
        for p, s in zip(gw.full_name, gw.season_num):
            find_player = overall[(overall.full_name == str(p)) & (overall.season_num == s-1)]
            if len(find_player) == 0:
                feature = float('NaN')
                feats.append(feature)
            else:
                feature = int(find_player[feat].values)
                feats.append(feature)
        gw[feature_name] = feats
    
    return gw
                

In [35]:
features_to_use_last_yr = ['assists', 'bonus', 'bps', 'clean_sheets','creativity','dreamteam_count','goals_conceded',
                          'goals_scored','ict_index','influence','minutes','own_goals','penalties_missed',
                           'penalties_saved', 'points_per_game','red_cards','saves', 'transfers_in','transfers_out',
                          'yellow_cards']

gw = get_last_season_overall(historical_team_df,players, features_to_use_last_yr)

assists
bonus
bps
clean_sheets
creativity
dreamteam_count
goals_conceded
goals_scored
ict_index
influence
minutes
own_goals
penalties_missed
penalties_saved
points_per_game
red_cards
saves
transfers_in
transfers_out
yellow_cards


In [36]:
players.to_csv('players_f.csv')
all_gws_df.to_csv('all_gws_f.csv')
gw.to_csv('all_with_history.csv')